<a href="https://colab.research.google.com/github/anmaxwell/UniNotebooks/blob/master/KTtrainWorking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ktrain

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [4]:
data_train = pd.read_csv('/content/train.csv')
data_train.head()
data_train.shape

(4928, 3)

In [45]:

data_test = pd.read_csv('/content/newtest.csv')
data_test.tail()

,id,comment_text,toxic
1903,a0d0f12b99e50ba1,"""\n\nI said it because you reverted some other...",0
1904,a0d191ffa3777b3b,REDIRECT Talk:College of Basic Science and Hum...,0
1905,a0d1e66228918fcb,"Oh, I thought you were an Indian Pariah Dog fa...",0
1906,a0d23cb7e695db30,"""\n\n """"The black flag of North Vietnam""""? \n\...",0
1907,a0d2e302918727ca,its funny how Zora keeps over looking this tra...,0


In [6]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(train_df = data_train,
                   text_column = 'comment_text',
                   label_columns=['toxic'],
                   val_df = data_test,
                   maxlen = 128,
                   preprocess_mode = 'bert')

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [7]:
X_train[0].shape

(4928, 128)

In [8]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc=preprocess)

Is Multi-Label? False
maxlen is 128
done.


In [9]:
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, y_train),
                             val_data = (X_test, y_test),
                             batch_size = 32)

In [10]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 4928 samples, validate on 1908 samples
4928/4928 [==============================] - 7360s 1s/sample - loss: 0.2280 - accuracy: 0.9123 - val_loss: 0.1149 - val_accuracy: 0.9575


In [11]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [28]:
data3 = ['I hate you so much right now']

In [19]:
predictor.predict(data3)

['1']

In [20]:
data4 = ['you are one of the most amazing people i ever met']

In [21]:
predictor.predict(data4)

['0']

In [37]:
data5 = ['why do you not just go to hell']

In [38]:
predictor.predict(data5)

['1']

In [39]:
# Saving Json file
model_json = learner.model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [40]:
# Saving Weights
learner.model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [44]:
# Save the model for later use
predictor.save("./toxicbert")